<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/%EB%AA%A8%EB%8D%B8%EB%B3%84%20RMSE%20%ED%99%95%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

biniroun_drug_data_path = kagglehub.dataset_download('biniroun/drug-data')

print('Data source import complete.')


In [ ]:
# Kaggle 노트북 상단에 RDKit 설치 명령어 추가
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 53.8 MB/s eta 0:00:00:00:0100:01


In [ ]:
# 필요한 경우 다른 라이브러리도 함께 업그레이드/설치
!pip install --upgrade scikit-learn pandas numpy xgboost lightgbm catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 77.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 83.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 73.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    

In [ ]:
# ✅ 1. 라이브러리 임포트
import pandas as pd
import numpy as np
import os
import warnings
import time

# RDKit 라이브러리
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors # Descriptors 추가 임포트

# Scikit-learn 라이브러리
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge # Stacking 메타 모델
from sklearn.metrics import mean_squared_error

# 부스팅 모델
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor # LightGBM 추가
from catboost import CatBoostRegressor

# 스태킹 앙상블
from sklearn.ensemble import StackingRegressor

# 경고 메시지 무시 및 RDKit 에러 메시지 억제
warnings.filterwarnings('ignore')
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')

print(f"--- Kaggle 파이프라인 시작: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())} ---")


--- Kaggle 파이프라인 시작: 2025-07-01 07:25:14 ---


In [ ]:
# ✅ 2. 데이터 경로 설정 및 데이터 로드 (Kaggle 전용 경로)
# 이 경로는 Kaggle에서 데이터셋을 추가했을 때의 기본 경로입니다.
# 만약 대회 데이터셋 이름이 다르다면, data_path를 변경해야 합니다.
# 예: data_path = "/kaggle/input/dacon-cyp3a4-inhibition-prediction"
data_path = "/kaggle/input/drug-data" # 가정된 데이터셋 경로

print(f"데이터 로딩 중... 경로: {data_path}")
try:
    train_df = pd.read_csv(f"{data_path}/train.csv")
    test_df = pd.read_csv(f"{data_path}/test.csv")
    submission_df = pd.read_csv(f"{data_path}/sample_submission.csv").copy() # .copy()를 통해 안전하게 작업
    print("데이터 로드 완료.")
except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. Kaggle 데이터셋 경로를 다시 확인해주세요: {data_path}")
    raise # 파일이 없으면 실행 중단

# 컬럼명 앞뒤 공백 제거 (데이터 클리닝)
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
submission_df.columns = submission_df.columns.str.strip()

print(f"훈련 데이터 Shape: {train_df.shape}")
print(f"테스트 데이터 Shape: {test_df.shape}")

데이터 로딩 중... 경로: /kaggle/input/drug-data
데이터 로드 완료.
훈련 데이터 Shape: (1681, 3)
테스트 데이터 Shape: (100, 2)


In [ ]:
# ✅ 3. 특성 추출 (Feature Engineering): 모든 RDKit Descriptors + Morgan Fingerprints
def get_all_molecular_features(smiles_string, n_bits=2048, radius=2):
    """
    SMILES 문자열로부터 모든 RDKit Descriptors와 Morgan Fingerprint를 추출합니다.
    """
    mol = Chem.MolFromSmiles(str(smiles_string))
    if mol is None:
        # 분자 생성 실패 시, 모든 피처를 0으로 채운 배열 반환
        num_descriptors = len(Descriptors._descList)
        return np.zeros(num_descriptors + n_bits, dtype=float)

    # RDKit Descriptors 추출
    descriptor_values = []
    for desc_name, desc_func in Descriptors._descList:
        try:
            val = desc_func(mol)
            # NaN 또는 Inf 값 처리: 0으로 대체 (모델이 NaN/Inf를 처리하지 못할 경우)
            if np.isnan(val) or np.isinf(val):
                descriptor_values.append(0.0)
            else:
                descriptor_values.append(val)
        except:
            # 계산 중 오류 발생 시 0으로 대체
            descriptor_values.append(0.0)

    # Morgan Fingerprint 추출
    try:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        morgan_fp_values = np.array(fp, dtype=float)
    except:
        # Fingerprint 계산 오류 시 0으로 채운 배열 반환
        morgan_fp_values = np.zeros(n_bits, dtype=float)

    # 두 피처 세트를 결합하여 반환
    return np.concatenate([descriptor_values, morgan_fp_values])

print("\n분자 특성 추출 시작 (모든 RDKit Descriptors + Morgan Fingerprints)...")
start_time_fe = time.time()

# 훈련 및 테스트 데이터셋에 피처 추출 함수 적용
train_features_raw = train_df['Canonical_Smiles'].apply(get_all_molecular_features)
test_features_raw = test_df['Canonical_Smiles'].apply(get_all_molecular_features)

# numpy 배열로 변환
X = np.vstack(train_features_raw.values)
X_test = np.vstack(test_features_raw.values)

# 타겟 변수 log1p 변환 (RMSE 개선에 도움)
y = np.log1p(train_df['Inhibition'])

end_time_fe = time.time()
print(f"특성 추출 완료. 소요 시간: {(end_time_fe - start_time_fe):.2f} 초")
print(f"특성 추출 후 훈련 데이터 피처 형태: {X.shape}")
print(f"특성 추출 후 테스트 데이터 피처 형태: {X_test.shape}")


분자 특성 추출 시작 (모든 RDKit Descriptors + Morgan Fingerprints)...
특성 추출 완료. 소요 시간: 25.63 초
특성 추출 후 훈련 데이터 피처 형태: (1681, 2256)
특성 추출 후 테스트 데이터 피처 형태: (100, 2256)


In [ ]:
# ==============================================================================
# --- 여기서부터 제안된 보완점 적용 ---
# ==============================================================================

# ✅ 4. 데이터 스케일링
print("\n데이터 스케일링 시작 (StandardScaler)...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
print("데이터 스케일링 완료.")

# ✅ 5. 피처 선택 (VarianceThreshold) - 스케일링 후 적용
from sklearn.feature_selection import VarianceThreshold

print("\n피처 선택 시작 (VarianceThreshold)...")
# 분산이 0인 (모든 값이 동일한) 피처만 제거. 스케일링 후에도 분산이 0이면 정보가 없음.
# threshold를 0.01 등으로 설정하여 분산이 매우 낮은 피처도 제거할 수 있으나,
# 일단 0으로 시작하여 완전히 불필요한 피처만 제거하는 것이 안전합니다.
selector = VarianceThreshold(threshold=0.0)
X_final = selector.fit_transform(X_scaled)
X_test_final = selector.transform(X_test_scaled)

print(f"피처 선택 전 훈련 데이터 형태: {X_scaled.shape}")
print(f"피처 선택 후 훈련 데이터 형태: {X_final.shape}")
print(f"제거된 피처 수: {X_scaled.shape[1] - X_final.shape[1]}")
print("피처 선택 완료.")

# 이제부터 모델 학습에는 X_final과 X_test_final을 사용합니다.
# y는 log1p 변환된 상태로 사용합니다.
# (이 코드 스니펫에는 모델 학습 부분이 포함되어 있지 않으므로, 다음 셀에서 이 변수들을 사용해야 합니다.)


데이터 스케일링 시작 (StandardScaler)...
데이터 스케일링 완료.

피처 선택 시작 (VarianceThreshold)...
피처 선택 전 훈련 데이터 형태: (1681, 2256)
피처 선택 후 훈련 데이터 형태: (1681, 2241)
제거된 피처 수: 15
피처 선택 완료.


In [ ]:
# ==============================================================================
# --- Subsequent Steps: Model Training, Stacking, Prediction, and Submission ---
# --- (This code assumes X_final, X_test_final, and y are already prepared) ---
# ==============================================================================

# Ensure necessary libraries are imported (from your initial setup)
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
# from sklearn.linear_model import Ridge
# import numpy as np
# import pandas as pd
# import time

# ✅ 5. 모델 학습 함수 (K-Fold 교차 검증 및 OOF/테스트 예측 생성)
# This function should already be defined in your notebook from previous steps.
# Re-including it here for context, but you don't need to define it again if it's already there.
def train_and_predict_kfolds(model, model_name, X_data, y_data, X_test_data, n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof_preds = np.zeros(len(X_data))
    test_preds = np.zeros(len(X_test_data))
    fold_rmses = []

    print(f"  {model_name} 학습 시작 (K-Fold CV)...")
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_data)):
        start_fold_time = time.time()
        X_train_fold, X_val_fold = X_data[train_idx], X_data[val_idx]
        y_train_fold, y_val_fold = y_data.iloc[train_idx], y_data.iloc[val_idx] # y_data is pandas Series

        # For CatBoost, if y_train_fold is a Series, it works fine.
        # For others, numpy array conversion might be slightly faster but Series also works.
        model.fit(X_train_fold, y_train_fold)

        # OOF 예측 (로그 스케일)
        fold_val_preds = model.predict(X_val_fold)
        oof_preds[val_idx] = fold_val_preds

        # RMSE calculation (still in log scale for consistency with target 'y')
        fold_rmse = np.sqrt(mean_squared_error(y_val_fold, fold_val_preds))
        fold_rmses.append(fold_rmse)

        # Test data prediction (still in log scale)
        test_preds += model.predict(X_test_data) / kf.n_splits

        end_fold_time = time.time()
        print(f"    Fold {fold+1}/{n_splits} 완료. RMSE: {fold_rmse:.4f}, 소요 시간: {(end_fold_time - start_fold_time):.2f} 초")

    avg_rmse = np.mean(fold_rmses)
    print(f"  ✅ {model_name} K-Fold 평균 RMSE (로그 스케일): {avg_rmse:.4f}")
    return test_preds, oof_preds # Returns predictions in log scale

# ✅ 6. 기본 모델 정의 및 학습 (LGBM, XGBoost, CatBoost)
print("\n기본 모델 학습 시작...")

# LightGBM 모델 정의
lgbm_model = LGBMRegressor(n_estimators=700, learning_rate=0.02, max_depth=8, random_state=42, n_jobs=-1, verbose=-1)
# Pass X_final and X_test_final here
lgbm_test_preds, lgbm_oof_preds = train_and_predict_kfolds(lgbm_model, "LightGBM", X_final, y, X_test_final)

# XGBoost 모델 정의
xgb_model = XGBRegressor(n_estimators=700, learning_rate=0.02, max_depth=8, random_state=42, n_jobs=-1, tree_method='hist')
# Pass X_final and X_test_final here
xgb_test_preds, xgb_oof_preds = train_and_predict_kfolds(xgb_model, "XGBoost", X_final, y, X_test_final)

# CatBoost 모델 정의
cat_model = CatBoostRegressor(iterations=700, learning_rate=0.02, depth=9, random_state=42, verbose=0, thread_count=-1)
# Pass X_final and X_test_final here
cat_test_preds, cat_oof_preds = train_and_predict_kfolds(cat_model, "CatBoost", X_final, y, X_test_final)

print("\n기본 모델 학습 및 예측 완료.")

# ✅ 7. Stacking 앙상블 (메타 모델 학습)
print("\nStacking 메타 모델 학습 시작...")

# Meta-model input data: OOF predictions from base models
# Ensure the stacking input dimensions are correct (samples, num_base_models)
stacked_X = np.vstack([lgbm_oof_preds, xgb_oof_preds, cat_oof_preds]).T
# Meta-model prediction target: Test predictions from base models
stacked_test = np.vstack([lgbm_test_preds, xgb_test_preds, cat_test_preds]).T

# Meta-model definition (Ridge)
meta_model = Ridge(alpha=1.0) # You can try tuning this alpha, or use RidgeCV

# Train meta-model (y is still in log scale)
meta_model.fit(stacked_X, y)

# Final Stacking prediction (in log scale)
stacked_final_preds_log = meta_model.predict(stacked_test)
print("Stacking 메타 모델 학습 및 예측 완료.")

# ✅ 8. 예측값 역변환 및 클리핑
print("\n최종 예측값 역변환 및 클리핑...")
# Reverse log1p transformation
final_predictions_original_scale = np.expm1(stacked_final_preds_log)

# Clip predictions to be within 0 and 100
submission_df['Inhibition'] = np.clip(final_predictions_original_scale, 0, 100)
print("예측값 처리 완료.")

# ✅ 9. 제출 파일 저장 (Kaggle working directory)
output_path = "/kaggle/working/submission_final_stacking_all_features.csv"
submission_df.to_csv(output_path, index=False)

print(f"\n✅ 최종 제출 파일 저장 완료: {output_path}")
print("\n최종 제출 파일 미리보기:")
print(submission_df.head())

# (Optional) Final model performance on training data (for reference)
print("\n훈련 데이터에 대한 최종 Stacking 모델 성능 평가 (참고용):")
# Stacking model often uses OOF predictions for a more accurate in-sample performance measure
final_oof_preds_log = meta_model.predict(stacked_X)
final_oof_preds_original_scale = np.expm1(final_oof_preds_log)

train_true_original_scale = train_df['Inhibition'] # Original un-transformed target
train_mae = mean_squared_error(train_true_original_scale, np.clip(final_oof_preds_original_scale, 0, 100)) # MAE
train_rmse = np.sqrt(mean_squared_error(train_true_original_scale, np.clip(final_oof_preds_original_scale, 0, 100))) # RMSE

print(f"훈련 데이터 MAE (Original Scale): {train_mae:.4f}")
print(f"훈련 데이터 RMSE (Original Scale): {train_rmse:.4f}")

print(f"\n--- Kaggle 파이프라인 완료: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())} ---")


기본 모델 학습 시작...
  LightGBM 학습 시작 (K-Fold CV)...
    Fold 1/5 완료. RMSE: 1.2366, 소요 시간: 5.80 초
    Fold 2/5 완료. RMSE: 1.2920, 소요 시간: 5.15 초
    Fold 3/5 완료. RMSE: 1.2249, 소요 시간: 4.55 초
    Fold 4/5 완료. RMSE: 1.2298, 소요 시간: 4.55 초
    Fold 5/5 완료. RMSE: 1.2882, 소요 시간: 4.60 초
  ✅ LightGBM K-Fold 평균 RMSE (로그 스케일): 1.2543
  XGBoost 학습 시작 (K-Fold CV)...
    Fold 1/5 완료. RMSE: 1.2242, 소요 시간: 19.61 초
    Fold 2/5 완료. RMSE: 1.2528, 소요 시간: 17.82 초
    Fold 3/5 완료. RMSE: 1.2617, 소요 시간: 19.38 초
    Fold 4/5 완료. RMSE: 1.2042, 소요 시간: 18.12 초
    Fold 5/5 완료. RMSE: 1.2684, 소요 시간: 19.63 초
  ✅ XGBoost K-Fold 평균 RMSE (로그 스케일): 1.2423
  CatBoost 학습 시작 (K-Fold CV)...
    Fold 1/5 완료. RMSE: 1.1918, 소요 시간: 74.18 초
    Fold 2/5 완료. RMSE: 1.2539, 소요 시간: 75.10 초
    Fold 3/5 완료. RMSE: 1.2154, 소요 시간: 76.10 초
    Fold 4/5 완료. RMSE: 1.1800, 소요 시간: 74.66 초
    Fold 5/5 완료. RMSE: 1.2437, 소요 시간: 73.95 초
  ✅ CatBoost K-Fold 평균 RMSE (로그 스케일): 1.2170

기본 모델 학습 및 예측 완료.

Stacking 메타 모델 학습 시작...
Stacking 메타 모델 학습 및 예측 완료.